In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin


def get_all_sublinks(url, max_depth=None, visited=None):
    """
    Recursively retrieve all sublinks from a webpage.

    Args:
        url (str): The starting URL.
        max_depth (int, optional): Maximum depth of recursion. None means no limit.
        visited (set, optional): Set of visited URLs to avoid duplicates.

    Returns:
        set: A set of all sublinks found.
    """
    if visited is None:
        visited = set()

    if max_depth is not None and max_depth <= 0:
        return set()

    try:
        response = requests.get(url)
        if response.status_code == 200:
            visited.add(url)
            soup = BeautifulSoup(response.text, 'html.parser')
            sublinks = set()

            for link in soup.find_all('a', href=True):
                absolute_link = urljoin(url, link['href'])
                if absolute_link not in visited:
                    sublinks.add(absolute_link)
                    sublinks.update(get_all_sublinks(absolute_link, max_depth - 1, visited))

            return sublinks
    except Exception as e:
        print(f"An error occurred: {str(e)}")


# Example usage:
start_url = 'http://nobullconstructions.com.au'  # Replace with your desired starting URL
all_sublinks = get_all_sublinks(start_url, max_depth=1)  # Adjust the maximum depth as needed


In [ ]:
requests.get('http://www.nobullconstructions.com.au').status_code

In [ ]:
len(urls)

In [3]:
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from urllib.parse import urljoin
import xlsxwriter

# set headers
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.5',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1'
}
all_data = []

# import website and company name from excel file
urls = pd.read_excel('yellowpages5.xlsx')
urls = urls[['name','website']]
urls['text'] = ''
urls['sublinks'] = ''
urls['sublinkstext'] = ''
# scrape website text and store in dataframe
# something wrong with index 958
with requests.Session() as s:
            temp = 'grantleigh.com.au/'
            r = s.get('https://www.grantleigh.com.au/', verify=False, headers=headers)
            soup = BeautifulSoup(r.text, 'html.parser')
            text = soup.get_text()
            text = text.replace('\n', '').replace('\t', '')
            sublinks=[]
            for link in soup.find_all('a', href=True):
                absolute_link = urljoin('https://www.grantleigh.com.au/', link['href'])
                if absolute_link not in sublinks:
                    if temp in absolute_link:
                        sublinks.append(absolute_link)
                

In [4]:
print(sublinks)

['https://www.grantleigh.com.au/projects', 'https://www.grantleigh.com.au/gallery', 'https://www.grantleigh.com.au/testimonials', 'https://www.grantleigh.com.au/about-grantleigh-homes', 'https://www.grantleigh.com.au/architectural-design-drafting-servi', 'https://www.grantleigh.com.au/construction-rebuild-sydney', 'https://www.grantleigh.com.au/interior-fit-outs', 'https://www.grantleigh.com.au/renovation-extension-builder-sydney']


In [4]:
import requests
import json
import re
import pandas as pd
from bs4 import BeautifulSoup
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from urllib.parse import urljoin
import xlsxwriter
from timeout_decorator import timeout 
import pandas as pd

# set headers
headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.5',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:102.0) Gecko/20100101 Firefox/102.0',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1'
}
all_data = []

# import website and company name from excel file
urls = pd.read_excel('yellowpages5.xlsx')
urls = urls[['name','website']]
urls['text'] = ''
urls['sublinks'] = ''
urls['sublinkstext'] = ''
# scrape website text and store in dataframe

@timeout(600)
def read_domain(i, urls):
    temp = urls['website'][i].replace('https://www.','')
    r = s.get(urls['website'][i], verify=False, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    text = soup.get_text()
    text = text.replace('\n', '').replace('\t', '')
    sublinks=[]
    for link in soup.find_all('a', href=True):
        absolute_link = urljoin(urls['website'][i], link['href'])
        if absolute_link not in sublinks:
            if temp in absolute_link:
                sublinks.append(absolute_link)
                
    urls['text'][i] = text
    urls['sublinks'][i] = sublinks
    
    return urls




@timeout(300)
def read_sublinks(x, urls, i):
    t = s.get(x, verify=False, headers=headers)
    soup1 = BeautifulSoup(t.text, 'html.parser')
    text1 = soup1.get_text()
    text1 = text1.replace('\n', '').replace('\t', '')
    urls['sublinkstext'][i] = urls['sublinkstext'][i] + text1
    return urls


# something wrong with index 958
with requests.Session() as s:
    for i in range(len(urls)):
        try:
            read_domain(i, urls)
            for x in urls['sublinks'][i]:
                try:
                    read_sublinks(x, urls, i)
                except TimeoutError:
                    print('timeout')
                    print(x)
                    pass
                except:
                    print('error in sublink')
                    print(x)
                    pass
        except TimeoutError:
            print('timeout')
            print(i)
            pass
        except:
            print('error in main link')
            print(i)
            pass
    urls.to_excel('all_text_try.xlsx', engine='xlsxwriter')    

error in main link
0
error in main link
1
error in main link
2
error in main link
3
error in main link
4
error in main link
5
error in main link
6
error in main link
7
error in main link
8
error in main link
9
error in main link
10
error in main link
11
error in main link
12
error in main link
13
error in main link
14
error in main link
15
error in main link
16
error in main link
17
error in main link
18
error in main link
19
error in main link
20
error in main link
21
error in main link
22
error in main link
23
error in main link
24
error in main link
25
error in main link
26
error in main link
27
error in main link
28
error in main link
29
error in main link
30
error in main link
31
error in main link
32
error in main link
33
error in main link
34
error in main link
35
error in main link
36
error in main link
37
error in main link
38
error in main link
39
error in main link
40
error in main link
41
error in main link
42
error in main link
43
error in main link
44
error in main link
4

In [5]:
@timeout(600)
def read_domain(i, urls):
    temp = urls['website'][i].replace('https://www.','')
    r = s.get(urls['website'][i], verify=False, headers=headers)
    soup = BeautifulSoup(r.text, 'html.parser')
    text = soup.get_text()
    text = text.replace('\n', '').replace('\t', '')
    sublinks=[]
    for link in soup.find_all('a', href=True):
        absolute_link = urljoin(urls['website'][i], link['href'])
        if absolute_link not in sublinks:
            if temp in absolute_link:
                sublinks.append(absolute_link)
                
    urls['text'][i] = text
    urls['sublinks'][i] = sublinks
    
    return urls

In [6]:
urls = pd.read_excel('yellowpages5.xlsx')

In [7]:
read_domain(0, urls)

AttributeError: module 'signal' has no attribute 'SIGALRM'

In [1]:
urls.to_excel('all_text_6.xlsx', engine='xlsxwriter')

NameError: name 'urls' is not defined